### Performance reports

##### DO NOT USE THIS!!!

#### Need to change

  - apply DB Schema to code

In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is not running!')

In [ ]:
## Some library
from address_convertor import pubkey_to_address, get_pubkey

In [ ]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [ ]:
# preparation database
import sqlite3

db_path = 'cluster.db'
conn = sqlite3.connect(db_path)
cur = conn.cursor()

    
def create_txhash_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxHash (
                     txhash TEXT PRIMARY KEY,
                     height INTEGER NOT NULL);
                ''')

def create_txin_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxIn (
                     address TEXT NOT NULL,
                     txhash TEXT NOT NULL);
                ''')

def create_txout_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS TxOut (
                     address TEXT NOT NULL,
                     txhash TEXT NOT NULL);
                ''')    

def create_cluster_table():
    cur.execute('''CREATE TABLE IF NOT EXISTS Cluster (
                     address TEXT PRIMARY KEY,
                     number INTEGER NOT NULL);
                ''')

    
def insert_txhash(txhash, height):
    cur.execute('''INSERT OR REPLACE INTO TxHash (
                       txhash, height) VALUES (
                       ?, ?);
                    ''', (txhash, height))
    
def insert_txin(address, txhash):
    cur.execute('''INSERT OR REPLACE INTO TxIn (
                       address, txhash) VALUES (
                       ?, ?);
                    ''', (address, txhash))

def insert_txout(address, txhash):
    cur.execute('''INSERT OR REPLACE INTO TxOut (
                       address, txhash) VALUES (
                       ?, ?);
                    ''', (address, txhash))

def insert_cluster(address, number):
    cur.execute('''INSERT OR IGNORE INTO Cluster (
                       address, number) VALUES (
                       ?, ?);
                    ''', (address, number))

    
def do_cluster(addresses, number):
    cur.executemany(f'''UPDATE Cluster SET number = {number} WHERE address IN ({",".join(addresses)})''')

def begin_transactions():
    cur.execute('BEGIN TRANSACTION;')

def commit_transactions():
    cur.execute('COMMIT;')
    
def get_max_height():
    cur.execute('''SELECT MAX(height) FROM TxHash;''')
    return cur.fetchone()[0]

def get_cluster_number(addresses):
    cur.execute(f'''SELECT MIN(number) FROM Cluster WHERE address IN ({",".join(addresses)})''')
    return cur.fetchone()[0]    

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
timeout = 300

def get_rpc():
    return AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)

rpc_connection = get_rpc()
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

In [ ]:
create_txin_table()
create_txout_table()
create_cluster_table()
create_txhash_table()

In [ ]:
def get_addresses(theight):
    txes = list()
    iaddresses = list()
    oaddresses = list()
    iaddrs = list()
    oaddrs = list()
    block_hash = rpc_connection.getblockhash(theight)
    block = rpc_connection.getblock(block_hash, 2)
    for tx in block['tx']:
        txes.append((tx['hash'], theight))
        for vin in tx['vin']:
            if 'coinbase' in vin.keys():
                iaddrs.extend(['coinbase'])
                continue
            ptx = rpc_connection.getrawtransaction(vin['txid'], 1)
            pvout = ptx['vout'][vin['vout']]
            if pvout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                 'witness_v0_keyhash', 'witness_v0_scripthash',
                                                 'multisig'):
                iaddrs.extend(pvout['scriptPubKey']['addresses'])
            elif pvout['scriptPubKey']['type'] in ('pubkey', 'nonstandard'):
                try:
                    iaddrs.extend([pubkey_to_address(get_pubkey(pvout['scriptPubKey']['hex']))])
                except UnboundLocalError:
                    iaddrs.extend([f'{vin["txid"]}{vin["vout"]}'])
            else:
                raise SystemExit(f'BUG!! {theight}\n{tx}\n{vin}')
        for addr in set(iaddrs):
            iaddresses.append((addr, tx['hash']))
        n = -1
        for vout in tx['vout']:
            n = n + 1
            if vout['scriptPubKey']['type'] in ('pubkeyhash', 'scripthash', 
                                                'witness_v0_keyhash', 'witness_v0_scripthash',
                                                'multisig'):
                oaddrs.extend(vout['scriptPubKey']['addresses'])
            elif vout['scriptPubKey']['type'] in ('pubkey', 'nonstandard'):
                try:
                    oaddrs.extend([pubkey_to_address(get_pubkey(vout['scriptPubKey']['hex']))])
                except UnboundLocalError:
                    oaddrs.extend([f'{tx["hash"]}{n}'])
            else:
                raise SystemExit(f'BUG!! {theight}\n{tx}\n{vout}')
        for addr in set(oaddrs):
            oaddresses.append((addr, tx['hash']))
    return txes, iaddresses, oaddresses

In [ ]:
import time
import pickle
import itertools
import multiprocessing

if os.path.exists('taking.pickle'):
    with open('taking.pickle', 'rb') as f:
        taking = pickle.load(f)
else:
    taking = dict()

term = 1000
start_height = get_max_height()
if start_height is None or start_height < term:
    start_height = 0
else:
    start_height = start_height - term
end_height = best_block['height']-term # default
print(f'Start from {start_height} to {end_height}')

rpc_connection = get_rpc()
pool_num = multiprocessing.cpu_count()
for sheight, eheight in zip(range(start_height, end_height, term), range(start_height+term, end_height+term, term)):
    stime = time.time()
    begin_transactions()
    if eheight >= end_height:
        eheight = end_height+1
    with multiprocessing.Pool(pool_num) as p:
        results = p.imap_unordered(get_addresses, range(sheight, eheight))
        for txes, iaddresses, oaddresses in results:
            for txhash, theight in txes:
                insert_txhash(txhash, theight)
            for address, txhash in iaddresses:
                insert_txin(address, txhash)
            for address, txhash in oaddresses:
                insert_txout(address, txhash)
                insert_cluster(address, -1) # -1 means single address cluster
    commit_transactions()
    etime = time.time()
    taking[sheight] = etime-stime
    print(f'Job done from {sheight} to {eheight} during {etime-stime}')

In [ ]:
with open('taking.pickle', 'wb') as f:
    pickle.dump(taking, f)
conn.close()